### Computing PCA using RDDs

###  PCA

The vectors that we want to analyze have length, or dimension, of 365, corresponding to the number of 
days in a year.

We want to perform [Principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis)
on these vectors. There are two steps to this process:

1. Computing the covariance matrix: this is a  simple computation. However, it takes a long time to compute and it benefits from using an RDD because it involves all of the input vectors.
2. Computing the eigenvector decomposition. this is a more complex computation, but it takes a fraction of a second because the size to the covariance matrix is $365 \times 365$, which is quite small. We do it on the head node usin `linalg`

### Computing the covariance matrix
Suppose that the data vectors are the column vectors denoted $x$ then the covariance matrix is defined to be
$$
E(x x^T)-E(x)E(x)^T
$$

Where $x x^T$ is the **outer product** of $x$ with itself.

If the data that we have is $x_1,x_2,x_n$ then the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

### `nan`s in arithmetic operations
* We store all of the measurements as single `bytearray` in a single column. Instead of using 365 columns.
* Why?
  1. Because serializing and desirializing is faster that way.
  1. Because numpy treats `nan` entries correctly:
      * In `numpy.nansum` `5+nan=5` while in dataframes `5+nan=nan`

In [1]:
import numpy as np
X=np.array([1,1,1,2])
print 'mean of',X,'=',np.mean(X)
print 'nanmean of',X,'=',np.nanmean(X)
X=np.array([1,1,np.NaN,2])
print 'mean of',X,'=',np.mean(X)
print 'nanmean of',X,'=',np.nanmean(X)

mean of [1 1 1 2] = 1.25
nanmean of [1 1 1 2] = 1.25
mean of [  1.   1.  nan   2.] = nan
nanmean of [  1.   1.  nan   2.] = 1.33333333333


#### When should you not use `np.nanmean` ?
Using `n.nanmean` is equivalent to assuming that choice of which elements to remove is independent of the values of the elements. 
* Example of bad case: suppose the larger elements have a higher probability of being `nan`. In that case `np.nanmean` will under-estimate the mean

#### Computing Cov matrix on vectors with NaNs
As it happens, we often get vectors $x$ in which some, but not all, of the entries are `nan`. 
Suppose that we want to compute the mean of the elements of $x$. If we use `np.mean` we will get the result `nan`. A useful alternative is to use `np.nanmean` which removes the `nan` elements and takes the mean of the rest.

#### Computing the covariance  when there are `nan`s
The covariance is a mean of outer products.

If the data that we have is $x_1,x_2,x_n$ then the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

In [2]:
x1=np.array([1,np.NaN,3,4,5])
x2=np.array([2,3,4,np.NaN,6])
stacked=np.array([np.outer(x1,x1),np.outer(x2,x2)])
stacked

array([[[  1.,  nan,   3.,   4.,   5.],
        [ nan,  nan,  nan,  nan,  nan],
        [  3.,  nan,   9.,  12.,  15.],
        [  4.,  nan,  12.,  16.,  20.],
        [  5.,  nan,  15.,  20.,  25.]],

       [[  4.,   6.,   8.,  nan,  12.],
        [  6.,   9.,  12.,  nan,  18.],
        [  8.,  12.,  16.,  nan,  24.],
        [ nan,  nan,  nan,  nan,  nan],
        [ 12.,  18.,  24.,  nan,  36.]]])

In [3]:
np.nanmean(stacked,axis=0)

/Users/jc/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: RuntimeWarning: Mean of empty slice
  if __name__ == '__main__':


array([[  2.5,   6. ,   5.5,   4. ,   8.5],
       [  6. ,   9. ,  12. ,   nan,  18. ],
       [  5.5,  12. ,  12.5,  12. ,  19.5],
       [  4. ,   nan,  12. ,  16. ,  20. ],
       [  8.5,  18. ,  19.5,  20. ,  30.5]])

### Loading Data into Dataframe

In [4]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
## add after creating spark_PCA.py
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/computeStats.py','lib/spark_PCA.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [5]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
# add after creating spark_PCA.py
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

### Climate data

The data we will use here comes from [NOAA](https://www.ncdc.noaa.gov/). Specifically, it was downloaded from This [FTP site](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/).

There is a large variety of measurements from all over the world, from 1870 will 2012.
in the directory `../../Data/Weather` you will find the following useful files:

* data-source.txt: the source of the data
* ghcnd-readme.txt: A description of the content and format of the data
* ghcnd-stations.txt: A table describing the Meteorological stations.



### Data cleaning

* Most measurements exists only for a tiny fraction of the stations and years. We therefor restrict our use to the following measurements:
```python
['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']
```

* 8 We consider only measurement-years that have at most 50 `NaN` entries

* We consider only measurements in the continential USA

* We partition the stations into 256 geographical rectangles, indexed from BBBBBBBB to SSSSSSSS. And each containing about 12,000 station,year pairs.

In [6]:
file_index='BBBBSSBB'
filename='US_Weather_%s.csv.gz'%file_index
data_dir='../../Data/Weather'
del_files='%s/%s*'%(data_dir,filename[:-3])
print del_files
!rm $del_files
command="curl https://mas-dse-open.s3.amazonaws.com/Weather/small/%s > %s/%s"%(filename,data_dir,filename)
print command
!$command
!ls -lh $data_dir/$filename

../../Data/Weather/US_Weather_BBBBSSBB.csv*
curl https://mas-dse-open.s3.amazonaws.com/Weather/small/US_Weather_BBBBSSBB.csv.gz > ../../Data/Weather/US_Weather_BBBBSSBB.csv.gz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3857k  100 3857k    0     0  2739k      0  0:00:01  0:00:01 --:--:-- 2937k
-rw-r--r--  1 jc  staff   3.8M May 12 23:40 ../../Data/Weather/US_Weather_BBBBSSBB.csv.gz


In [7]:
!gunzip $data_dir/$filename
filename=data_dir+'/US_Weather_BBBBSSBB.csv'
!ls -lh $filename

-rw-r--r--  1 jc  staff    13M May 12 23:40 ../../Data/Weather/US_Weather_BBBBSSBB.csv


In [8]:
import pickle
List=pickle.load(open(filename,'rb'))
len(List)

12884

In [9]:
df=sqlContext.createDataFrame(List)
print df.count()
df.show(5)

12884
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|    106.7| 43.4622| -76.4933|       TMAX|USC00306314|     0|[20 50 40 4C 80 4...|1926.0|BBBBSSBB|
|    106.7| 43.4622| -76.4933|       TMAX|USC00306314|     0|[00 00 A0 D3 00 0...|1927.0|BBBBSSBB|
|    106.7| 43.4622| -76.4933|       TMAX|USC00306314|     1|[80 4D 80 D4 E0 D...|1928.0|BBBBSSBB|
|    106.7| 43.4622| -76.4933|       TMAX|USC00306314|     1|[00 53 00 00 00 C...|1929.0|BBBBSSBB|
|    106.7| 43.4622| -76.4933|       TMAX|USC00306314|     2|[E0 54 E0 55 00 4...|1930.0|BBBBSSBB|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
only showing top 5 rows



In [10]:
sqlContext.registerDataFrameAsTable(df,'weather') #using older sqlContext instead of newer (V2.0) sparkSession

In [11]:
from time import time
t=time()

N=sc.defaultParallelism
print 'Number of executors=',N
print 'took',time()-t,'seconds'

Number of executors= 3
took 0.00095796585083 seconds


### Computing PCA for each measurement

In [12]:
measurements=['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']

#### Homework
The code below computes the covariance matrix using RDDs.
The code allows undefined entries and calculates the covariance without bias.

Your homework is to complete the missing parts in the code (Marked with `...`) so that it would calculate the covariance correctly.

In [13]:
Query="SELECT station,count(distinct year), sum(undefs) "\
      "FROM weather group by station having sum(undefs) < 1000 "\
      "and count(distinct year)>50 "\
      "and count(distinct measurement) = 6"
print Query
df1 = sqlContext.sql(Query)
print df1.show()

SELECT station,count(distinct year), sum(undefs) FROM weather group by station having sum(undefs) < 1000 and count(distinct year)>50 and count(distinct measurement) = 6
+-----------+--------------------+-----------+
|    station|count(DISTINCT year)|sum(undefs)|
+-----------+--------------------+-----------+
|USC00306314|                  86|        968|
|USC00303851|                  87|        737|
|USC00306062|                  67|        655|
|USC00305925|                  59|        984|
+-----------+--------------------+-----------+

None


In [14]:
from numpy import linalg as LA
STAT={}  # dictionary storing the statistics for each measurement
Clean_Tables={}

for meas in measurements:
    t=time()
    Query="SELECT * FROM weather\n\tWHERE measurement ='{0}' ".format(meas)
    print Query
    df1 = sqlContext.sql(Query)
    data=df1.rdd.map(lambda row: unpackArray(row['vector'],np.float16))
    #get basic statistics
    STAT[meas]=computeOverAllDist(data)   # Compute the statistics 

    # compute covariance matrix
    OUT=computeCov(data)

    #find PCA decomposition
    eigval,eigvec=LA.eig(OUT['Cov'])

    # collect all of the statistics in STAT[meas]
    STAT[meas]['eigval']=eigval
    STAT[meas]['eigvec']=eigvec
    STAT[meas].update(OUT)

    print 'time for',meas,'is',time()-t

SELECT * FROM weather
	WHERE measurement ='TMAX' 
shape of E= (365,) shape of NE= (365,)
time for TMAX is 23.6996071339
SELECT * FROM weather
	WHERE measurement ='SNOW' 
shape of E= (365,) shape of NE= (365,)
time for SNOW is 25.0931401253
SELECT * FROM weather
	WHERE measurement ='SNWD' 
shape of E= (365,) shape of NE= (365,)
time for SNWD is 18.0825810432
SELECT * FROM weather
	WHERE measurement ='TMIN' 
shape of E= (365,) shape of NE= (365,)
time for TMIN is 23.4352359772
SELECT * FROM weather
	WHERE measurement ='PRCP' 
shape of E= (365,) shape of NE= (365,)
time for PRCP is 33.5417699814
SELECT * FROM weather
	WHERE measurement ='TOBS' 
shape of E= (365,) shape of NE= (365,)
time for TOBS is 12.4890291691


In [15]:
from pickle import dump
filename=data_dir+'/STAT_%s.pickle'%file_index
dump((STAT,STAT_Descriptions),open(filename,'wb'))
!ls -lrth $data_dir

total 601160
-rw-r--r--    1 jc  staff   218B May  1 08:46 data-source.txt
-rw-r--r--    1 jc  staff   2.1M May  1 08:46 stations.pkl.gz
-rw-r--r--    1 jc  staff   270B May  1 08:46 ghcnd-version.txt
-rw-r--r--    1 jc  staff   7.4M May  1 08:46 ghcnd-stations_buffered.txt
-rw-r--r--    1 jc  staff   8.0M May  1 08:46 ghcnd-stations.txt
-rw-r--r--    1 jc  staff    22K May  1 08:46 ghcnd-readme.txt
drwxr-xr-x  404 jc  staff    13K May  1 08:46 decon_BBBSBBBB_TOBS.parquet
drwxr-xr-x  404 jc  staff    13K May  1 08:46 decon_BBBSBBBB.parquet
drwxr-xr-x   10 jc  staff   340B May  1 08:46 US_Weather_BBBSBBBB.parquet
-rw-r--r--    1 jc  staff    16M May  1 08:46 STAT_BBBSBBBB.pickle.gz
drwxr-xr-x  404 jc  staff    13K May  5 22:39 decon_BBBSBBBB_SNWD.parquet
drwxr-xr-x  404 jc  staff    13K May  6 14:39 decon_BBBSBBBB_PRCP.parquet
-rw-r--r--    1 jc  staff    13M May  8 06:43 US_Weather_BBBSBBBB.csv
-rw-r--r--    1 jc  staff    78M May  8 06:50 STAT_BBBSBBBB.pickle
drwxr-xr-x  404 jc  staff

In [16]:
filename='%s/US_Weather_%s.parquet'%(data_dir,file_index)
!rm -rf $filename
df.write.save(filename)